In [1]:
import pandas as pd
import numpy as np
import json
import nltk
import re
import csv
import matplotlib.pyplot as plt 
import seaborn as sns
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [2]:
meta = pd.read_csv("../data/movie.metadata.tsv", sep = '\t', header = None)
meta.columns = ["movie_id",1,"movie_name",3,4,5,6,7,"genre"]
meta.head()

,movie_id,1,movie_name,3,4,5,6,7,genre
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}"


In [3]:
plots = []
with open("../data/plot_summaries.txt", 'r') as f:
    reader = csv.reader(f, dialect='excel-tab') 
    for row in tqdm(reader):
        plots.append(row)

42303it [00:01, 38211.60it/s]


In [4]:
movie_id = []
plot = []

# extract movie Ids and plot summaries
for i in tqdm(plots):
    movie_id.append(i[0])
    plot.append(i[1])

# create dataframe
movies = pd.DataFrame({'movie_id': movie_id, 'plot': plot})

100%|██████████| 42303/42303 [00:00<00:00, 1192954.10it/s]


In [5]:
movies.shape

(42303, 2)

In [6]:
# change datatype of 'movie_id'
meta['movie_id'] = meta['movie_id'].astype(str)

# merge meta with movies
movies = pd.merge(movies, meta[['movie_id', 'movie_name', 'genre']], on = 'movie_id')

In [7]:
movies.shape

(42204, 4)

In [8]:
genres = [] 

# extract genres
for i in movies['genre']: 
    genres.append(list(json.loads(i).values())) 

# add to 'movies' dataframe  
movies['genre_new'] = genres

In [9]:
movies.head()

,movie_id,plot,movie_name,genre,genre_new
0,23890098,"Shlykov, a hard-working taxi driver and Lyosha...",Taxi Blues,"{""/m/07s9rl0"": ""Drama"", ""/m/03q4nz"": ""World ci...","[Drama, World cinema]"
1,31186339,The nation of Panem consists of a wealthy Capi...,The Hunger Games,"{""/m/03btsm8"": ""Action/Adventure"", ""/m/06n90"":...","[Action/Adventure, Science Fiction, Action, Dr..."
2,20663735,Poovalli Induchoodan is sentenced for six yea...,Narasimham,"{""/m/04t36"": ""Musical"", ""/m/02kdv5l"": ""Action""...","[Musical, Action, Drama, Bollywood]"
3,2231378,"The Lemon Drop Kid , a New York City swindler,...",The Lemon Drop Kid,"{""/m/06qm3"": ""Screwball comedy"", ""/m/01z4y"": ""...","[Screwball comedy, Comedy]"
4,595909,Seventh-day Adventist Church pastor Michael Ch...,A Cry in the Dark,"{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D...","[Crime Fiction, Drama, Docudrama, World cinema..."


In [10]:
movies_new = movies[~(movies['genre_new'].str.len() == 0)]

In [11]:
movies_new.shape

(41793, 5)

In [ ]:
all_genres = sum(genres,[])

In [ ]:
genre_set = set(all_genres)

In [ ]:
all_genres = nltk.FreqDist(all_genres) 

# create dataframe
all_genres_df = pd.DataFrame({'Genre': list(all_genres.keys()), 'Count': list(all_genres.values())})

In [ ]:
g = all_genres_df.nlargest(columns="Count", n = 70) 
plt.figure(figsize=(15,15)) 
ax = sns.barplot(data=g, x= "Count", y = "Genre") 
ax.set(ylabel = 'Count') 
plt.show()

In [12]:
movies_new[movies_new['movie_id']=='1952976']

,movie_id,plot,movie_name,genre,genre_new
6,1952976,"{{plot}} The film opens in 1974, as a young gi...",Dark Water,"{""/m/01jfsb"": ""Thriller"", ""/m/07s9rl0"": ""Drama...","[Thriller, Drama, Horror]"


In [ ]:
def clean_text(text):
    # remove a string like {{plot}}
    text = re.sub("\s*{{\w*}}\s*", "", text)
    # remove backslash-apostrophe 
    text = re.sub("\'", "", text)
    return text

In [ ]:
movies_new['clean_plot'] = movies_new['plot'].apply(lambda x: clean_text(x))

In [41]:
pd.set_option('display.max_colwidth', 300)
movies_new[movies_new['movie_id']=='1952976']

,movie_id,plot,movie_name,genre,genre_new
6,1952976,"{{plot}} The film opens in 1974, as a young girl, Dahlia, stands outside after school in the rain, waiting for her mother. Flash forward to 2005, we see a grown-up Dahlia in the midst of a bitter mediation with ex-husband, Kyle , over custody of their daughter, Cecilia . Kyle wants Cecilia to l...",Dark Water,"{""/m/01jfsb"": ""Thriller"", ""/m/07s9rl0"": ""Drama"", ""/m/03npn"": ""Horror""}","[Thriller, Drama, Horror]"


In [ ]:
def freq_words(x, terms):
    all_words = ' '.join([text for text in x]) 
    all_words = all_words.split() 
    fdist = nltk.FreqDist(all_words) 
    words_df = pd.DataFrame({'word':list(fdist.keys()), 'count':list(fdist.values())}) 
  
    # selecting top 20 most frequent words 
    d = words_df.nlargest(columns="count", n = terms) 
  
    # visualize words and frequencies
    plt.figure(figsize=(12,15)) 
    ax = sns.barplot(data=d, x= "count", y = "word") 
    ax.set(ylabel = 'Word') 
    plt.show()

# print 100 most frequent words 
freq_words(movies_new['clean_plot'], 100)

In [ ]:
dict={}
count=0
for movie_id in movies_new['movie_id']:
    plot_sr = movies_new[movies_new['movie_id']==movie_id]['clean_plot']
    for str_obj in plot_sr:
        plot=str_obj
    sentence_list = []
    for sentence in plot.split('.'):
        if len(sentence)==0:
            count+=1

In [29]:
x = movies_new[movies_new['movie_id']=='31186339']['plot']

In [30]:
for y in x:
    plot = y

In [37]:
from nltk.tokenize import sent_tokenize
z1=sent_tokenize(plot)

In [32]:
z1

['The nation of Panem consists of a wealthy Capitol and twelve poorer districts.',
 'As punishment for a past rebellion, each district must provide a boy and girl  between the ages of 12 and 18 selected by lottery  for the annual Hunger Games.',
 'The tributes must fight to the death in an arena; the sole survivor is rewarded with fame and wealth.',
 'In her first Reaping, 12-year-old Primrose Everdeen is chosen from District 12.',
 'Her older sister Katniss volunteers to take her place.',
 "Peeta Mellark, a baker's son who once gave Katniss bread when she was starving, is the other District 12 tribute.",
 'Katniss and Peeta are taken to the Capitol, accompanied by their frequently drunk mentor, past victor Haymitch Abernathy.',
 'He warns them about the "Career" tributes who train intensively at special academies and almost always win.',
 'During a TV interview with Caesar Flickerman, Peeta unexpectedly reveals his love for Katniss.',
 'She is outraged, believing it to be a ploy to ga

In [33]:
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
z2=sent_detector.tokenize(plot.strip())

In [35]:
for w in z1:
    if len(w)==0:
        print('--------')

In [39]:
plot.strip()==plot

True

In [40]:
type(z1)

list